In [1]:
!nvidia-smi

Sat Aug 22 12:34:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!python -m pip install git+https://github.com/ShivamShrirao/dnn_from_scratch.git

  Cloning https://github.com/ShivamShrirao/dnn_from_scratch.git to /tmp/pip-req-build-37ix8dpj
  Running command git clone -q https://github.com/ShivamShrirao/dnn_from_scratch.git /tmp/pip-req-build-37ix8dpj
  Created wheel for dnn-from-scratch: filename=dnn_from_scratch-0.1.dev1-cp36-none-any.whl size=39847 sha256=a3660c994598da163f8a0cf87e9885c2c37abf65faa345e0e76dffd0d98caa25
  Stored in directory: /tmp/pip-ephem-wheel-cache-97uok4qh/wheels/33/77/6c/1140f6295368e7045f3b49fdda98e7f07b2513b8b55093ca7b
Successfully built dnn-from-scratch


In [4]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning_from_scratch'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 266 (delta 169), reused 181 (delta 84), pack-reused 0
Receiving objects: 100% (266/266), 210.40 KiB | 6.57 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [5]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning_from_scratch


In [6]:
!git pull origin master

From https://github.com/ShivamShrirao/deep_Q_learning_from_scratch
 * branch            master     -> FETCH_HEAD
Already up to date.


In [7]:
import gym
import time
import cv2
import numpy as np
from collections import deque

from settings import *
from agent import *
from experience import *

In [8]:
def get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=3):
    model=Sequential()
    model.add(Conv2D(num_kernels=16, kernel_size=3, stride=(2, 2), activation=functions.relu, input_shape=input_shape))
    model.add(Conv2D(num_kernels=32, kernel_size=3, stride=(2, 2), activation=functions.relu))
    model.add(Conv2D(num_kernels=64, kernel_size=3, stride=(2, 2), activation=functions.relu))
    model.add(Conv2D(num_kernels=128, kernel_size=3, stride=(2, 2), activation=functions.relu))
    model.add(Flatten())
    model.add(Dense(256, activation=functions.relu))
    model.add(Dense(no_of_actions, activation=functions.echo))

    model.compile(optimizer=optimizers.adam, loss=functions.mean_squared_error, learning_rate=0.0001)
    return model

def preproc_obsv(obsv):
    obsv = cv2.cvtColor(obsv, cv2.COLOR_RGB2GRAY)
    obsv = obsv[34:194:2,::2]
    return obsv

In [10]:
agt = DQN_Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=2e-6, target_update_thresh=1000)
agt.model = get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=len(agt.actions))
agt.target = get_model(input_shape=(HEIGHT,WIDTH,NFRAMES), no_of_actions=len(agt.actions))
agt.update_target()
agt.model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 16)         relu             592
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 32)         relu             4640
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 64)         relu             18496
__________________________________________________________________________________________
3 Conv2D(Conv2D)          (None, 5, 5, 128)          relu             73856
__________________________________________________________________________________________
4 F

In [ ]:
D_exp = ReplayMemory(capacity=1_500_000, nlap=1)

In [ ]:
D_exp.current_state.nbytes/1024/1024/1024

8.940696716308594

In [ ]:
env = gym.make('Breakout-v4')       # v4 to ensure ball is fired when required

In [ ]:
D_exp.len

1111721

In [ ]:
for i_episode in range(2000):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    start = time.time()
    for t in range(10_000):
        s_s = time.time()
#         env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            action = agt.get_action(state_que)
        next_observation, reward, done, info = env.step(action)
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -1
            fire = True
        ep_score += reward
        if action==1:
            action = 0
        D_exp.store_transition(state, agt.actions.index(action), reward, done)
        observation = next_observation

        if (D_exp.len-D_exp.idx_len) > BATCH_SIZE:
            grads = agt.train(D_exp, BATCH_SIZE)
        print('\r', t, action, ep_score, end='  ')
        if done:
            break
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
    if not i_episode%8:
        agt.model.save_weights("/content/drive/My Drive/breakout.w8s")
env.close()

In [ ]:
agt.epsilon = 0

In [ ]:
from gym import wrappers

In [ ]:
wrapped_env = wrappers.Monitor(env.env, '/content/videos/' + str(time.time()) + '/')
for i_episode in range(1):
    obinit = wrapped_env.reset()
    if not i_episode:
        observation = obinit
        state = preproc_obsv(observation)
        state_que = deque([], maxlen=NFRAMES)
        for i in range(NFRAMES):
            state_que.append(state)
    ep_score = 0
    lives = 5
    fire = True
    preds = []
    reward_history = []
    start = time.time()
    t=-1
    while 1:
        t+=1
        s_s = time.time()
#         wrapped_env.render()
        state = preproc_obsv(observation)
        state_que.append(state)
        if fire:
            action = 1
            fire = False
        else:
            out = agt.predict(state_que)
            pidx = cp.argmax(out[0]).item()
            preds.append(out[0][pidx].item())
            action = agt.actions[pidx]
        next_observation, reward, done, info = wrapped_env.step(action)
        if lives != info['ale.lives']:
            lives = info['ale.lives']
            reward = -1
            fire = True
        ep_score += reward
        reward_history.append(reward)
        if action==1:
            action = 0
        observation = next_observation
        print('\r', t, ep_score, end='  ')
        if done:
            break
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
wrapped_env.close()

Episode 1 finished after 994 timesteps, Score: 19.0, Epsilon: 0.000000, Time: 2.53


In [ ]:
!rm /content/videos/* -rf